In [ ]:
#!pip install -q kaggle

In [ ]:
'''from google.colab import files
files.upload()'''

In [ ]:
'''!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json
!kaggle competitions download -c tuberculosis-drug-resistance-prediction-pza'''

  0% 0.00/74.1k [00:00<?, ?B/s]
100% 74.1k/74.1k [00:00<00:00, 28.3MB/s]
  0% 0.00/91.0k [00:00<?, ?B/s]
100% 91.0k/91.0k [00:00<00:00, 29.4MB/s]
  0% 0.00/443k [00:00<?, ?B/s]
100% 443k/443k [00:00<00:00, 61.4MB/s]
  0% 0.00/4.43k [00:00<?, ?B/s]
100% 4.43k/4.43k [00:00<00:00, 4.23MB/s]


In [ ]:
'''!unzip ./X_trainData_1.csv.zip
!rm ./X_trainData_1.csv.zip'''

Archive:  ./X_trainData_1.csv.zip
  inflating: X_trainData_1.csv       


In [ ]:
import pandas as pd
df = pd.read_csv('X_trainData_1.csv')
#df

In [ ]:
df1 = pd.read_csv('Y_trainData_1.csv')
#df1

In [ ]:
for i in df1.columns:
  print(df1[i].value_counts())

 1    2057
 0    1278
-1      58
Name: RIF, dtype: int64
 1    1832
 0    1524
-1      37
Name: INH, dtype: int64
 1    2246
 0     695
-1     452
Name: PZA, dtype: int64
 1    2346
 0     973
-1      74
Name: EMB, dtype: int64
-1    1312
 1    1061
 0    1020
Name: STR, dtype: int64
-1    2214
 1     942
 0     237
Name: CIP, dtype: int64
-1    2054
 1     752
 0     587
Name: CAP, dtype: int64
-1    2033
 1    1127
 0     233
Name: AMK, dtype: int64
-1    2056
 1    1070
 0     267
Name: MOXI, dtype: int64
-1    2703
 1     603
 0      87
Name: OFLX, dtype: int64
-1    2110
 1    1013
 0     270
Name: KAN, dtype: int64


In [ ]:
df = pd.read_csv('X_trainData_1.csv')
df1 = pd.read_csv('Y_trainData_1.csv')

#Dropping this columns as these columns contain -1.
df.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
df.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
df.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)
count=0

#Dropping the training data whose corresponding target variable is -1.
for ind in df1.index:
  x = df1['PZA'][ind]
  if x == -1:
    count+=1
    df.drop(index=ind,inplace=True)
    df1.drop(index=ind,inplace=True)
print(count)
df['PZA'] = df1['PZA']
df

452


,DEL_CF_410280_d918T_307_iniB,DEL_CF_4408101_d102C_34_gid,DEL_F_4408101_d101C_gid_G34G,INS_CF_4242820_i2957G_986_embC,INS_F_409772_i410ATCT_iniB_G137G,INS_F_4247020_i506CC_embB_G169G,INS_F_4247970_i1456GT_embB_G486G,INS_N_4243642_i409GTCCCGGGGCGCCAC_embA_S137S,INS_P_3074519_G.117_thyA,INS_P_3074521_G.115_thyA,SNP_CN_1673449_A10C_T4P_fabG1,SNP_CN_1674263_T62C_I21T_inhA,SNP_CN_1674481_T280G_S94A_inhA,SNP_CN_1674952_C751G_P251A_inhA,SNP_CN_2102891_A152G_F51S_ndh,SNP_CN_2102990_A53G_V18A_ndh,SNP_CN_2154613_TC_katG_Q500R,SNP_CN_2154724_C1388A_R463L_katG,SNP_CN_2155167_GT_katG_S315R,SNP_CN_2155168_C944G_S315T_katG,SNP_CN_2155168_C944T_S315N_katG,SNP_CN_2155276_CT_katG_G279D,SNP_CN_2289040_AC_pncA_W68G,SNP_CN_2289090_TC_pncA_H51R,SNP_CN_2289099_TG_pncA_K48T,SNP_CN_2289180_A62C_V21G_pncA,SNP_CN_2289202_A40G_C14R_pncA,SNP_CN_2518836_AC_kasA_E241A,SNP_CN_2518839_CT_kasA_A242V,SNP_CN_2518919_G805A_G269S_kasA,SNP_CN_2519048_G934A_G312S_kasA,SNP_CN_2726051_G37A_L13F_oxyR',SNP_CN_2726338_T146G_V49G_ahpC,SNP_CN_2726338_TGGT146-149GGGG_VV49-50GG_ahpC,SNP_CN_2726350_G158T_W53L_ahpC,SNP_CN_2726409_GC_ahpC_D73H,SNP_CN_2726737_A545C_D182A_ahpC,SNP_CN_3073805_CA_thyA_E223*,SNP_CN_3073868_T604C_T202A_thyA,SNP_CN_3074465_GC_thyA_P3A,...,pncA_C_snp,eis_NC_indel,ethA_C_snp,katG_F_indel,iniB_F_indel,gyrA-gyrB_NC_snp,eis_C_snp,iniA-iniB-iniC_NC_snp,embA-embB_NC_snp,rpoB_F_indel,alr_C_snp,rpsA_NF_indel,ddl_C_snp,embB_C_snp,rpsL_NC_snp,embA_C_snp,kasA_C_snp,rpoB_C_snp,iniC_C_snp,gyrB_C_snp,iniA_F_indel,eis_NF_indel,kasA_NF_indel,ndh_NC_snp,rrl_F_indel,pncA_F_indel,fabG1-inhA_NC_snp,rpsA_NC_snp,embA_F_indel,thyA_C_snp,ahpC_NC_snp,ahpC_C_snp,gid_C_snp,iniA_C_snp,gyrB_F_indel,embC_C_snp,eis_NC_snp,rpsA_C_snp,rrl-rrs_NC_snp,PZA
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
## Correlation
#Trying to find the correlation between the feature columns and the target variable for Dimensionality Reduction.Didn't workout
'''
import seaborn as sns
import matplotlib.pyplot as plt
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")
correlation = corrmat['PZA'].sort_values()
correlation = correlation.iloc[:10]
df = df.drop(columns=correlation.index,axis=1)'''

In [ ]:
#Sampling the dataset to make a test set containing 20% 1 labels and 20% 0 labels.Saving these test set in test.csv

'''from sklearn.utils import resample
df['PZA'] = df1['PZA']
df_majority = df[df['PZA']==1]
df_minority = df[df['PZA']==0]

test_data = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=459,     # to match minority class
                                 random_state=42) # reproducible results

test_data1 = resample(df_minority, 
                                 replace=False,    # sample without replacement
                                 n_samples=69,     # to match minority class
                                 random_state=42)
                                  # reproducible results

test = test_data.append(test_data1)
test.to_csv('test.csv',index=None)
df_majority = df_majority.drop(test_data.index)
df_minority = df_minority.drop(test_data1.index)'''


In [ ]:
#Breaking the dataset into 3 parts for ensemble method
'''df_majority_downsampled1 = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=626,     # to match minority class
                                 random_state=1) # reproducible results
temp = pd.DataFrame()
temp = temp.append(df_majority_downsampled1)
df_majority_downsampled2 = resample(df_majority.drop(temp.index), 
                                 replace=False,    # sample without replacement
                                 n_samples=626,     # to match minority class
                                 random_state=1) # reproducible results

temp = temp.append(df_majority_downsampled2)
df_majority_downsampled3 = df_majority.drop(temp.index)

df_majority_downsampled1.shape,df_majority_downsampled2.shape,df_majority_downsampled3.shape'''

((626, 220), (626, 220), (535, 220))

In [ ]:
#Making the dataset for the 3 classifiers containing approximately equal amount of data of label '1' and label '0
'''traindata1 = pd.DataFrame()
traindata1 = traindata1.append(df_majority_downsampled1)
traindata1 = traindata1.append(df_minority)
y1 = traindata1['PZA']
traindata1.drop(['PZA'],axis=1,inplace=True)

traindata2 = pd.DataFrame()
traindata2 = traindata2.append(df_majority_downsampled2)
traindata2 = traindata2.append(df_minority)
y2 = traindata2['PZA']
traindata2.drop(['PZA'],axis=1,inplace=True)

traindata3 = pd.DataFrame()
traindata3 = traindata3.append(df_majority_downsampled3)
traindata3 = traindata3.append(df_minority)
y3 = traindata3['PZA']
traindata3.drop(['PZA'],axis=1,inplace=True)'''

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
import numpy as np
import pickle

In [ ]:
#Making the train and test split of the whole dataset for trying the different ML models
ss = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)

traindata1 = df.drop(columns=['PZA'],axis=1)
y1 = df['PZA']

for train_index, test_index in ss.split(traindata1,y1):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = traindata1.iloc[train_index], traindata1.iloc[test_index]
  y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]

X_train.shape,X_test.shape

((2646, 219), (295, 219))

In [ ]:
y_train.value_counts()

1    2021
0     625
Name: PZA, dtype: int64

In [ ]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.01,0.03,0.05, 0.10, 0.15, 0.20, 0.25, 0.30,0.35,0.4] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350,370,400]   
}
params1 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9,2,2.2,2.4,2.6,2.8,3,3.2,3.4],
   'kernel' : ['linear','rbf','sigmoid'],
    'class_weight':['balanced',''],
  'decision_function_shape':['ovo', 'ovr']
}
params2 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9],
    'class_weight':['balanced',''],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter':[20,40,60,80,100,120,140,180,200,230,250,270,300]
}
params3 = {
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12,14, 15,16,18,20],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350]   
}

In [ ]:
classifier= RandomForestClassifier(random_state=0)
random_search=RandomizedSearchCV(classifier,param_distributions=params2,n_iter=15,scoring='roc_auc',
                                 n_jobs=-1,cv=2,verbose=3)

grid_search=GridSearchCV(classifier,param_grid=params3,scoring='roc_auc',
                                 n_jobs=-1,cv=2,verbose=3)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
print(grid_search.best_estimator_)#This prints out the classifier with its optimal hyperparameters for the given dataset. 
grid_search.best_score_#This prints out the best score the classifer can get on the given data.

In [ ]:
lr = LogisticRegression(C=1.7, class_weight='balanced',max_iter=20,
                    solver='saga')
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9016949152542373
[[ 65   5]
 [ 24 201]]
              precision    recall  f1-score   support

           0       0.73      0.93      0.82        70
           1       0.98      0.89      0.93       225

    accuracy                           0.90       295
   macro avg       0.85      0.91      0.88       295
weighted avg       0.92      0.90      0.91       295



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
svm = SVC(C=0.5,class_weight='',probability=True)
svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9254237288135593
[[ 59  11]
 [ 11 214]]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84        70
           1       0.95      0.95      0.95       225

    accuracy                           0.93       295
   macro avg       0.90      0.90      0.90       295
weighted avg       0.93      0.93      0.93       295



In [ ]:
rf = RandomForestClassifier(n_estimators=55,max_depth=15,random_state=0)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9288135593220339
[[ 60  10]
 [ 11 214]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.85        70
           1       0.96      0.95      0.95       225

    accuracy                           0.93       295
   macro avg       0.90      0.90      0.90       295
weighted avg       0.93      0.93      0.93       295



In [ ]:
xf = XGBClassifier(n_estimators=50,max_depth=4,learning_rate=0.1)
                  
xf.fit(X_train, y_train)
print(xf.score(X_test, y_test))

y_pred = xf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9389830508474576
[[ 58  12]
 [  6 219]]
              precision    recall  f1-score   support

           0       0.91      0.83      0.87        70
           1       0.95      0.97      0.96       225

    accuracy                           0.94       295
   macro avg       0.93      0.90      0.91       295
weighted avg       0.94      0.94      0.94       295



**Trying Ensemble Method**

In [ ]:
tempm1 = pd.DataFrame()
tempm1 = tempm1.append(df_majority)
tempm1 = tempm1.append(df_minority)
ylab1 = tempm1['PZA']
tempm1.drop(['PZA'],axis=1,inplace=True)

test1 = pd.read_csv('test.csv')
ytest1 = test1['PZA']
test1 = test1.drop(['PZA'],axis=1)

svm1 = SVC(gamma='auto',C=2,kernel='linear')
svm1.fit(traindata1, y1)

svm2 = SVC(gamma='auto',C=0.8,kernel='linear')
svm2.fit(traindata2, y2)

svm3 = SVC(gamma='auto',C=1,kernel='linear',class_weight='balanced')
svm3.fit(traindata3, y3)

feature1 = svm1.predict(tempm1)
feature2 = svm2.predict(tempm1)
feature3 = svm3.predict(tempm1)

feature_set = pd.DataFrame()
feature_set['fea1'] = feature1
feature_set['fea2'] = feature2
feature_set['fea3'] = feature3

final_svm = LogisticRegression(solver='liblinear',multi_class='auto',max_iter=300,C=1,class_weight='balanced')
final_svm.fit(feature_set, ylab1)

testfeature1 = svm1.predict(test1)
testfeature2 = svm2.predict(test1)
testfeature3 = svm3.predict(test1)

testfeature_set = pd.DataFrame()
testfeature_set['fea1'] = testfeature1
testfeature_set['fea2'] = testfeature2
testfeature_set['fea3'] = testfeature3


y_pred1 = final_svm.predict(testfeature_set)

print(confusion_matrix(ytest1,y_pred1))
print(classification_report(ytest1,y_pred1))


[[ 64   5]
 [ 39 420]]
              precision    recall  f1-score   support

           0       0.62      0.93      0.74        69
           1       0.99      0.92      0.95       459

    accuracy                           0.92       528
   macro avg       0.80      0.92      0.85       528
weighted avg       0.94      0.92      0.92       528



**Trying Neural Net**

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.utils import class_weight
from keras import optimizers
tf.__version__


'2.3.0'

In [ ]:
!pip freeze > requirement.txt

In [ ]:
y_train.value_counts()

NameError: ignored

In [ ]:
#!rm -r ./model
!rm -r ./model1

In [ ]:
#This code is for model training
model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=219),
tf.keras.layers.Dense(128,activation='relu'),
#tf.keras.layers.Dropout(0.1),
#tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(16,activation='relu'),
tf.keras.layers.Dense(4,activation='relu'),
tf.keras.layers.Dense(1,activation='sigmoid'),
])


In [ ]:
##This code is for model training
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),
              )
model.summary()
weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(weights)
temp = {0:weights[0],1:weights[1]}
type(temp)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               28160     
_________________________________________________________________
dense_9 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 68        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 5         
Total params: 30,297
Trainable params: 30,297
Non-trainable params: 0
_________________________________________________________________
[2.1168     0.65462642]


dict

In [ ]:
#Load the pre-trained model by this code
#!unzip model_PZA.zip
#!rm model_PZA.zip
model = tf.keras.models.load_model('./model_PZA/')
model.summary()

Archive:  model_PZA.zip
   creating: model/
   creating: model/assets/
  inflating: model/saved_model.pb    
   creating: model/variables/
  inflating: model/variables/variables.data-00000-of-00001  
  inflating: model/variables/variables.index  
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 128)               28160     
_________________________________________________________________
dense_39 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_40 (Dense)             (None, 4)                 68        
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 5         
Total params: 30,297
Trainable params: 30,297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train,y_train,epochs=3,validation_data=(X_test,y_test))

83/83 [==============================] - 0s 4ms/step - loss: 0.1388 - val_loss: 0.2264


In [ ]:
model.save('model_PZA')
!zip -r model_PZA.zip ./model_PZA

INFO:tensorflow:Assets written to: model1/assets
  adding: model1/ (stored 0%)
  adding: model1/assets/ (stored 0%)
  adding: model1/variables/ (stored 0%)
  adding: model1/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: model1/variables/variables.index (deflated 63%)
  adding: model1/saved_model.pb (deflated 88%)


In [ ]:
preds = model.predict(X_test)
preds1 = np.where(preds>0.5,1,0)
preds2 = np.where(model.predict(X_train)>0.5,1,0)
print(np.unique(preds1,return_counts=True))
print(accuracy_score(y_train,preds2))
print(accuracy_score(y_test,preds1))
print(confusion_matrix(y_test,preds1))
print(roc_auc_score(y_test,preds))

(array([0, 1]), array([ 59, 236]))
0.9584278155706727
0.9152542372881356
[[ 52  18]
 [  7 218]]
0.9540317460317462


updating: model1.dat (deflated 63%)
updating: model2.dat (deflated 63%)
updating: model3.dat (deflated 70%)


In [ ]:
X_testdata = pd.read_csv('X_testData_1.csv')
Y_testdata_nolabel = pd.read_csv('Y_testData_1_nolabels_PZA.csv')
output_id = set(Y_testdata_nolabel['ID'])
len(output_id)

773

In [ ]:
X_testdata.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)

generate_id = X_testdata['ID']
X_testdata.drop('ID',axis=1,inplace=True)
generate_id

0         1
1         2
2         3
3         4
4         5
       ... 
995     996
996     997
997     998
998     999
999    1000
Name: ID, Length: 1000, dtype: int64

In [ ]:
#Run this to produce output for ML models
output_pred = lr.predict_proba(X_testdata)
#print(output_pred[:5])
out = pd.DataFrame({'PZA':output_pred[:,1]})
out['ID'] = generate_id
out = out[['ID','PZA']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
out['PZA'].value_counts()

0.044709    30
0.021909    27
0.010402    22
0.052074    16
0.025275    11
            ..
0.086161     1
0.010458     1
0.073489     1
0.997742     1
0.068622     1
Name: PZA, Length: 538, dtype: int64

In [ ]:
#Run this to produce output for Neural network
output_pred = model.predict_proba(X_testdata)
#print(output_pred[:5])
out = pd.DataFrame({'PZA':output_pred[:,0]})
out['ID'] = generate_id
out = out[['ID','PZA']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
out['PZA'].value_counts()

In [ ]:
out.to_csv('Group_7_output.csv',index=None)